### check versions of key python libraries
#scipy 
import scipy
print('scipy: %s' % scipy.__version__)
#numpy
import numpy
print('numpy: %s' % numpy.__version__)
# matplotlib
import matplotlib
print('matplotlib: %s' % matplotlib.__version__)
# pandas
import pandas
print('pandas: %s' % pandas.__version__)
# statsmodels
import statsmodels
print('statsmodels: %s' % statsmodels.__version__)
# scikit-learn
import sklearn
print('sklearn: %s' % sklearn.__version__)
print ("\n hola")

# Vamos a cargar un data set de: 

https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-total-female-births.csv



In [1]:
# load dataset using read_csv()
from pandas import read_csv
series = read_csv('daily-total-female-births.csv', header=0, index_col=0, parse_dates=True,
squeeze=True)
print(type(series))
print(series.head())

<class 'pandas.core.series.Series'>
Date
1959-01-01    35
1959-01-02    32
1959-01-03    30
1959-01-04    31
1959-01-05    44
Name: Births, dtype: int64


In [2]:
print(series.head(10)) #Primeros 10 datos
# You can also use the tail() function to get the last n records of the dataset.
print(series.tail(10)) #Ultimos 10 datos
print(series.size) #total de datos
print(series['1959-01']) #Imprimimos los datos para enero
#vamos a ver las características de la serie:
print("Caracteristicas de la serie")
print(series.describe())


Date
1959-01-01    35
1959-01-02    32
1959-01-03    30
1959-01-04    31
1959-01-05    44
1959-01-06    29
1959-01-07    45
1959-01-08    43
1959-01-09    38
1959-01-10    27
Name: Births, dtype: int64
Date
1959-12-22    39
1959-12-23    40
1959-12-24    38
1959-12-25    44
1959-12-26    34
1959-12-27    37
1959-12-28    52
1959-12-29    48
1959-12-30    55
1959-12-31    50
Name: Births, dtype: int64
365
Date
1959-01-01    35
1959-01-02    32
1959-01-03    30
1959-01-04    31
1959-01-05    44
1959-01-06    29
1959-01-07    45
1959-01-08    43
1959-01-09    38
1959-01-10    27
1959-01-11    38
1959-01-12    33
1959-01-13    55
1959-01-14    47
1959-01-15    45
1959-01-16    37
1959-01-17    50
1959-01-18    43
1959-01-19    41
1959-01-20    52
1959-01-21    34
1959-01-22    53
1959-01-23    39
1959-01-24    32
1959-01-25    37
1959-01-26    43
1959-01-27    39
1959-01-28    35
1959-01-29    44
1959-01-30    38
1959-01-31    24
Name: Births, dtype: int64
Caracteristicas de la serie
count

# 5.4 Date Time Feature

https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv

In [11]:
# create date time features of a dataset
from pandas import read_csv
from pandas import DataFrame
#read_csv tambien nos regresa un data frame
series = read_csv('daily-min-temperatures.csv', header=0, 
                  index_col=0, parse_dates=True, squeeze=True)
print(f"series type: {type(series)}")
print("Series tail (3):")
print(series.tail(3))
### Utilizando el DataFrame de pandas separamos los elementos de fecha.
dataframe = DataFrame()
dataframe['year'] = [series.index[i].year for i in range(len(series))]
dataframe['month'] = [series.index[i].month for i in range(len(series))]
dataframe['day'] = [series.index[i].day for i in range(len(series))]
dataframe['temperature'] = [series[i] for i in range(len(series))]
print(dataframe.head(5))


series type: <class 'pandas.core.series.Series'>
Series tail (3):
Date
1990-12-29    13.5
1990-12-30    15.7
1990-12-31    13.0
Name: Temp, dtype: float64
   year  month  day  temperature
0  1981      1    1         20.7
1  1981      1    2         17.9
2  1981      1    3         18.8
3  1981      1    4         14.6
4  1981      1    5         15.8


Using just the month and day information alone to predict temperature is not sophisticated and will likely result in a poor model. Nevertheless, this information coupled with additional engineered features may ultimately result in a better model. You may enumerate all the properties of a time-stamp and consider what might be useful for your problem, such as:

* Minutes elapsed for the day.
* Hour of day.
* Business hours or not.
* Weekend or not.
* Season of the year.
* Business quarter of the year.
* Daylight savings or not.
* Public holiday or not.
* Leap year or not.



Date-time based features are a good start, but it is often a lot more useful to
include the values at previous time steps. These are called lagged values and we will look at
adding these features in the next section.

## 5.5 Lag Features

Lag features are the classical way that time series forecasting problems are transformed into supervised learning problems. The simplest approach is to predict the value at the next time (t+1) given the value at the current time (t). The supervised learning problem with shifted values looks as follows:
~~~
Value(t), Value(t+1)
Value(t), Value(t+1)
Value(t), Value(t+1)
~~~

The Pandas library provides the shift() function1 to help create these shifted or lag features from a time series dataset. Shifting the dataset by 1 creates the t column, adding a NaN (unknown) value for the first row. The time series dataset without a shift represents the t+1. 

Let’s make this concrete with an example. The first 3 values of the temperature dataset are 20.7, 17.9, and 18.8. The shifted and unshifted lists of temperatures for the first 3 observations are therefore:


In [20]:
from pandas import concat

temps = DataFrame(series.values)
dataframe = concat([temps.shift(1), temps], axis=1)
dataframe.columns = ['t', 't+1']
print(dataframe.head(5))


      t   t+1
0   NaN  20.7
1  20.7  17.9
2  17.9  18.8
3  18.8  14.6
4  14.6  15.8


You can see that we would have to discard the first row to use the dataset to train a supervised learning model, as it does not contain enough data to work with. The addition of lag features is called the sliding window method, in this case with a window width of 1. It is as though we are sliding our focus along the time series for each observation with an interest in only what is within the window width. 

We can expand the window width and include more lagged features. For example, below is the above case modified to include the last 3 observed values to predict the value at the next time step.


In [23]:
df2 = concat([temps.shift(3), temps.shift(2), temps.shift(1), temps], axis=1)
df2.columns = ['t-2', 't-1', 't', 't+1']
print("head:")
print(df2.head(5))
print("tail:")
print(df2.tail(5))

head:
    t-2   t-1     t   t+1
0   NaN   NaN   NaN  20.7
1   NaN   NaN  20.7  17.9
2   NaN  20.7  17.9  18.8
3  20.7  17.9  18.8  14.6
4  17.9  18.8  14.6  15.8
tail:
       t-2   t-1     t   t+1
3645  10.0  12.9  14.6  14.0
3646  12.9  14.6  14.0  13.6
3647  14.6  14.0  13.6  13.5
3648  14.0  13.6  13.5  15.7
3649  13.6  13.5  15.7  13.0


Again, you can see that we must discard the first few rows that do not have enough data to train a supervised model. A difficulty with the sliding window approach is how large to make the window for your problem. Perhaps a good starting point is to perform a sensitivity analysis and try a suite of different window widths to in turn create a suite of different views of your dataset and see which results in better performing models. There will be a point of diminishing returns.


## 5.6 Rolling Window Statistics p31
A step beyond adding raw lagged values is to add a summary of the values at previous time steps. We can calculate summary statistics across the values in the sliding window and include these as features in our dataset. Perhaps the most useful is the mean of the previous few values, also called the rolling mean. We can calculate the mean of the current and previous values and use that to predict the next value. For the temperature data, we would have to wait 3 time steps before we had 2 values to take the average of before we could use that value to predict a 3rd value. For example:

~~~
mean(t-1, t), t+1
mean(20.7, 17.9), 18.8
19.3, 18.8
~~~

Pandas provides a rolling() function3 that creates a new data structure with the window of values at each time step. We can then perform statistical functions on the window of values collected for each time step, such as calculating the mean. First, the series must be shifted.

Then the rolling dataset can be created and the mean values calculated on each window of two values. Here are the values in the first three rolling windows:

~~~
#,  Window Values
1,  NaN
2,  NaN, 20.7
3, 20.7, 17.9
~~~
This suggests that we will not have usable data until the 3rd row. Finally, as in the previous section, we can use the concat() function to construct a new dataset with just our new columns. 

The example below demonstrates how to do this with Pandas with a window size of 2.


In [28]:
shifted = temps.shift(1)
window = shifted.rolling(window=2)
means = window.mean()
dataframe = concat([means, temps], axis=1)
dataframe.columns = ['mean(t-1,t)', 't+1']
print(dataframe.head(5))


   mean(t-1,t)   t+1
0          NaN  20.7
1          NaN  17.9
2        19.30  18.8
3        18.35  14.6
4        16.70  15.8


Running the example prints the first 5 rows of the new dataset. We can see that the first two rows are not useful.

* The first NaN was created by the shift of the series.
* The second because NaN cannot be used to calculate a mean value.
* Finally, the third row shows the expected value of 19.30 (the mean of 20.7 and 17.9) used to predict the 3rd value in the series of 18.8.

There are more statistics we can calculate and even different mathematical ways of calculating the definition of the window. Below is another example that shows a window width of 3 and a dataset comprised of more summary statistics, specifically the minimum, mean, and maximum value in the window.

You can see in the code that we are explicitly specifying the sliding window width as a named variable. This allows us to use it both in calculating the correct shift of the series and in specifying the width of the window to the rolling() function. 

In this case, the window width of 3 means we must shift the series forward by 2 time steps. This makes the first two rows NaN. Next, we need to calculate the window statistics with 3 values per window. It takes 3 rows before we even have enough data from the series in the window to start calculating statistics. The values in the first 5 windows are as follows:


In [30]:
width = 3
shifted = temps.shift(width - 1)
window = shifted.rolling(window=width)
dataframe = concat([window.min(), window.mean(), window.max(), temps], axis=1)
dataframe.columns = ['min', 'mean', 'max', 't+1']
print(dataframe.head(10))


    min       mean   max   t+1
0   NaN        NaN   NaN  20.7
1   NaN        NaN   NaN  17.9
2   NaN        NaN   NaN  18.8
3   NaN        NaN   NaN  14.6
4  17.9  19.133333  20.7  15.8
5  14.6  17.100000  18.8  15.8
6  14.6  16.400000  18.8  15.8
7  14.6  15.400000  15.8  17.4
8  15.8  15.800000  15.8  21.8
9  15.8  16.333333  17.4  20.0
